In [1]:
import pandas as pd

In [2]:
jcols = ['StateCode', 'Type', 'County', 'UnitIdentifier', 'AnotherGov',
        'ItemCode', 
        'ThousandsOfDollars',
        'Year',
        'ImputationType']
jcols2 = ['IDCode', 'IDName', 'County Name', 'StateCode', 'CountyCode',
         'PlaceCode', 'Population', 'Population Year', 'Enrollment', 'Enrollment Year',
         'Function Code for Special Districts', 'School Level Code', 'Fiscal Year Ending',
         'Survey Year']

jyears = list(range(2013, 2016))
jyears2 = ['08', '09', '10', '11', '12']

for x in jyears:
    subdf = pd.read_fwf(f'{x}full.txt', 
                 header=None,widths=[2,1,3,3,5,3,12,4,1],
                 names=jcols,
                dtype = 'str')
    if x == 2013:
        subdf2 = subdf
    else:
        subdf2 = pd.concat([subdf, subdf2])

jdf = subdf2
#jdf = pd.read_fwf('2016full.txt', 
#                 header=None,widths=[2,1,3,3,5,3,12,4,1],
#                 names=jcols,
#                dtype = 'str')
jdf2 = pd.read_fwf('GID.txt', header=None,
                  widths=[14,64,35,2,3,5,9,2,7,2,2,2,4,2],
                  names = jcols2,
                 dtype = 'str')


states = pd.read_csv('states.csv', dtype = 'str')

In [41]:
jdf2[jdf2['County Name'] == 'Clark']

,County Name,IDName,State/County
724,Clark,CLARK COUNTY,05019
4426,Clark,CLARK COUNTY,16033
6096,Clark,CLARK COUNTY,18019
7973,Clark,CLARK COUNTY,21049
13045,Clark,CLARK COUNTY,32003
16120,Clark,CLARK COUNTY,39023
19686,Clark,CLARK COUNTY,46025
22585,Clark,CLARK COUNTY,53011


In [42]:
states

,State Name,Abbreviation,StateCode
0,Alabama,AL,01
1,Alaska,AK,02
2,Arizona,AZ,03
3,Arkansas,AR,04
4,California,CA,05
5,Colorado,CO,06
6,Connecticut,CT,07
7,Delaware,DE,08
8,District of Columbia,DC,09
9,Florida,FL,10


In [3]:
for x in range(0,5):       #Pre 2013 data is in a totally different format
    jdf5 = pd.read_csv(f'IndFin{jyears2[x]}a.Txt', dtype = 'str')
    jdf6 = pd.read_csv(f'IndFin{jyears2[x]}b.Txt', dtype = 'str')
    jdf7 = pd.read_csv(f'IndFin{jyears2[x]}c.Txt', dtype = 'str')        
    if x == 0:
        jdf8 = pd.concat([jdf5, jdf6, jdf7])
    else:
        jdf8 = pd.concat([jdf5, jdf6, jdf7, jdf8])
    print(x, jdf8.shape)

0 (75627, 554)
1 (149070, 554)
2 (222411, 554)
3 (295590, 554)
4 (566523, 554)


In [4]:
colsOfInterest = ['Year4', 'ID', 'State Code', 'County',
                  'LTD Out-Gen-Other NEC', 
                  'Property Tax', 'Individual Income Tax', 
                  'Corp Net Income Tax', 'Total Salaries & Wages']
#jdf8 = jdf8[colsOfInterest]

jdf['State/County'] = jdf['StateCode'] + jdf['County']
jdf8['State/County'] = jdf8['State Code'] + jdf8['County']

jdf8.rename(columns = {'Year4':'Year', 'ID':'IDCode'}, inplace = True)
jdf8.reset_index(drop = True, inplace = True)

In [5]:
for x in range(len(jdf2)):      #If IDCode[2] == 1, it's an entry at the county level
    jdf2['IDCode'][x] = jdf2['IDCode'][x][2]

for x in range(len(jdf8)):      #This takes very long, convert dataframe to spark first
    jdf8['IDCode'][x] = jdf8['IDCode'][x][2]
    if x % 20000 == 0:
        print(x)

0
20000
40000
60000
80000
100000
120000
140000
160000
180000
200000
220000
240000
260000
280000
300000
320000
340000
360000
380000
400000
420000
440000
460000
480000
500000
520000
540000
560000


In [6]:
jdf.drop(jdf[jdf['County'] == '000'].index, inplace = True)
jdf2.drop(jdf2[(jdf2['IDCode'] != '1')].index, inplace = True)

print(jdf8.shape)
jdf8.drop(jdf8[(jdf8['IDCode'] != '1')].index, inplace = True)
print(jdf8.shape)


(566523, 555)
(30333, 555)


In [7]:
jdf2['State/County'] = jdf2['StateCode'] + jdf2['CountyCode']
jdf2 = jdf2[['County Name', 'IDName', 'State/County']]

In [8]:
sub1 = jdf[jdf['ItemCode'] == 'T01'][['StateCode', 'County', 'ThousandsOfDollars', 'Year', 'State/County']]
sub2 = jdf[jdf['ItemCode'] == 'Z00'][['StateCode', 'County', 'ThousandsOfDollars', 'Year', 'State/County']]
sub3 = jdf[jdf['ItemCode'] == 'U20'][['StateCode', 'County', 'ThousandsOfDollars', 'Year', 'State/County']]
sub4 = jdf[jdf['ItemCode'] == 'U30'][['StateCode', 'County', 'ThousandsOfDollars', 'Year', 'State/County']]

sub1.rename(columns = {'ThousandsOfDollars':'Property Tax'}, inplace = True)
sub2.rename(columns = {'ThousandsOfDollars':'Total Salaries & Wages'}, inplace = True)
sub3.rename(columns = {'ThousandsOfDollars':'Interest Revenue'}, inplace = True)
sub4.rename(columns = {'ThousandsOfDollars':'Fines and Forfeits'}, inplace = True)


In [9]:
sub1['Property Tax'] = sub1['Property Tax'].astype(int)
sub1 = sub1.groupby(['State/County','Year'])['Property Tax'].agg('sum').reset_index()

sub2['Total Salaries & Wages'] = sub2['Total Salaries & Wages'].astype(int)
sub2 = sub2.groupby(['State/County','Year'])['Total Salaries & Wages'].agg('sum').reset_index()

sub3['Interest Revenue'] = sub3['Interest Revenue'].astype(int)
sub3 = sub3.groupby(['State/County','Year'])['Interest Revenue'].agg('sum').reset_index()

sub4['Fines and Forfeits'] = sub4['Fines and Forfeits'].astype(int)
sub4 = sub4.groupby(['State/County','Year'])['Fines and Forfeits'].agg('sum').reset_index()


In [10]:
master = sub1.merge(sub2, on = ['Year', 'State/County'], how = 'inner')
master = master.merge(sub3, on = ['Year', 'State/County'], how = 'inner')
master = master.merge(sub4, on = ['Year', 'State/County'], how = 'inner')

In [11]:
jdf9 = jdf8[['Year', 'Property Tax', 'Total Salaries & Wages', 'Interest Revenue', 'Fines and Forfeits', 'State/County']]

In [12]:
jdf9 = jdf9.dropna() #This is 2008-2012 data

In [13]:
master = pd.concat([jdf9, master])

In [14]:
#master = master.merge(jdf2, on = 'State/County')#, how = 'left') 
#master = master.merge(states, on = 'StateCode') #Gets state name and abbreviation
master.reset_index(drop = True,inplace = True)

In [15]:
master['StateCode'] = ""
for x in range(len(master)):
    master['StateCode'][x] = master['State/County'][x][0:2]

In [16]:
master = master.merge(jdf2, on = 'State/County') #Translates state/County to county names
master = master.merge(states, on = 'StateCode') #Gets state name and abbreviation

In [22]:
master2 = master[['Year', 'Property Tax', 'Total Salaries & Wages', 'Interest Revenue',
       'Fines and Forfeits', 'County Name', 'State Name']]

In [35]:
master2.rename(columns = {'County Name':'County', 'State Name':'State'}, inplace = True)


C:\Users\Jake Thompson\anaconda3\lib\site-packages\pandas\core\frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [37]:
master2.to_csv("census.csv")


In [36]:
master2

,Year,Property Tax,Total Salaries & Wages,Interest Revenue,Fines and Forfeits,County,State
0,2012,1822,2454,15,406,BALDWIN,ALABAMA
1,2013,3658,27567,66,4,BALDWIN,ALABAMA
2,2014,5011,35520,293,284,BALDWIN,ALABAMA
3,2015,5022,35842,300,263,BALDWIN,ALABAMA
4,2012,5994,4899,163,0,BLOUNT,ALABAMA
...,...,...,...,...,...,...,...
4046,2014,225446,98223,1328,728,ALFALFA,RHODE ISLAND
4047,2015,232601,101854,1611,771,ALFALFA,RHODE ISLAND
4048,2013,395641,221266,2228,2363,ATOKA,RHODE ISLAND
4049,2014,256141,125412,2647,1882,ATOKA,RHODE ISLAND
